### Import libraries

In [2]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import regularizers
import numpy as np
import cv2
import os

### Preprocess images

#### Convert images into grayscale, invert colors and dilate them

In [5]:
dataDir = "../data_copy/"

dirList = os.listdir(dataDir)

for directory in dirList:
    path = os.path.join(dataDir, directory)

    for fileName in os.listdir(path):
        imagePath = os.path.join(path, fileName)

        image = cv2.imread(imagePath)

        grayscale = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

        inverted = cv2.bitwise_not(grayscale)

        kernel = np.ones((3, 3), np.uint8)

        dilatedImage = cv2.dilate(inverted, kernel, iterations=1)

        folder = os.path.join(dataDir, directory)

        imagePath = folder + "/" + fileName

        cv2.imwrite(imagePath, dilatedImage)


### Load and split data

In [14]:
ds_train = keras.utils.image_dataset_from_directory(
    directory = "../data_copy/",
    labels='inferred',
    label_mode='categorical',
    color_mode="grayscale",
    batch_size=64,
    image_size=(45, 45),
    shuffle=True,
    seed=0,
    validation_split=0.2,
    subset="training"
)

ds_val = keras.utils.image_dataset_from_directory(
    directory = "../data_copy/",
    labels='inferred',
    label_mode='categorical',
    color_mode="grayscale",
    batch_size=64,
    image_size=(45, 45),
    shuffle=True,
    seed=0,
    validation_split=0.2,
    subset="validation"
)

ds_test = ds_val.take(145)
ds_val = ds_val.skip(145)

Found 185465 files belonging to 16 classes.
Using 148372 files for training.
Found 185465 files belonging to 16 classes.
Using 37093 files for validation.


In [15]:
print('Batches for testing -->', ds_test.cardinality())
print('Batches for validating -->', ds_val.cardinality())

Batches for testing --> tf.Tensor(145, shape=(), dtype=int64)
Batches for validating --> tf.Tensor(435, shape=(), dtype=int64)


### Create model

In [16]:

img_height = 45
img_width = 45
num_classes = 16
img_channels = 1

model = keras.Sequential([
  layers.Conv2D(16, 3, padding='same', activation='relu',input_shape=(img_height, img_width, img_channels)),
  layers.MaxPooling2D(),
  layers.Conv2D(32, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(64, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Dropout(0.2),
  layers.Flatten(),
  layers.Dense(128, activation='relu'),
  layers.Dense(num_classes, activation='softmax')
])

### Compile model

In [17]:
model.compile(optimizer="Adam", loss="categorical_crossentropy", metrics=["accuracy"])

### Train model

In [18]:
model.fit(ds_train, epochs=15, validation_data=ds_val)

Epoch 1/15
2319/2319 [==============================] - 10s 4ms/step - loss: 0.2101 - accuracy: 0.9557 - val_loss: 0.0579 - val_accuracy: 0.9812
Epoch 2/15
2319/2319 [==============================] - 9s 4ms/step - loss: 0.0573 - accuracy: 0.9820 - val_loss: 0.0412 - val_accuracy: 0.9861
Epoch 3/15
2319/2319 [==============================] - 9s 4ms/step - loss: 0.0465 - accuracy: 0.9848 - val_loss: 0.0347 - val_accuracy: 0.9887
Epoch 4/15
2319/2319 [==============================] - 9s 4ms/step - loss: 0.0394 - accuracy: 0.9872 - val_loss: 0.0314 - val_accuracy: 0.9895
Epoch 5/15
2319/2319 [==============================] - 9s 4ms/step - loss: 0.0339 - accuracy: 0.9889 - val_loss: 0.0252 - val_accuracy: 0.9909
Epoch 6/15
2319/2319 [==============================] - 9s 4ms/step - loss: 0.0298 - accuracy: 0.9904 - val_loss: 0.0210 - val_accuracy: 0.9931
Epoch 7/15
2319/2319 [==============================] - 9s 4ms/step - loss: 0.0271 - accuracy: 0.9910 - val_loss: 0.0198 - val_accuracy

### Evaluate model on test set

In [19]:
model.evaluate(ds_test)

145/145 [==============================] - 0s 3ms/step - loss: 0.0128 - accuracy: 0.9964


[0.012755334377288818, 0.9964439868927002]

### Save model

In [20]:
model.save("../models/model1")

2022-04-27 17:13:55.594677: W tensorflow/python/util/util.cc:368] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


INFO:tensorflow:Assets written to: ../models/model1/assets
